In [2]:
import glob
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np

a = 0
c = 0
cap = cv.VideoCapture('data/det_one_class/33/222.mp4')
ret = True
while ret:
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    #frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    laplacian = 255 * np.uint8(np.absolute(cv.Sobel(frame,cv.CV_64F,0,1,ksize=3))> 750)
    plt.imshow(laplacian, cmap = 'gray')
cap.release()

Can't receive frame (stream end?). Exiting ...


In [3]:

import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from typing import List
from PIL import Image
import os
import cv2 as cv
from tqdm import tqdm

def load_frames(path: str)->List[str]:
    cap = cv.VideoCapture(path)
    ret = True
    frames = []
    while ret:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    return frames

In [4]:
annot = [
    {"id": 1, "participants": [9, 11, 14, 12], "special":[]},
    {"id": 4, "participants": [28, 3], "special":[3]},
    {"id": 5, "participants": [5, 8], "special":[61]},
    {"id": 7, "participants": [9], "special":[58]},
    {"id":8, "participants": [17], "special":[37]},
    {"id":9, "participants": [41, 47], "special":[54]},
    {"id":10, "participants": [19, 23], "special":[73, 85], "timing":{23:(0, 5)}},
    {"id":12, "participants": [12, 25], "special":[110]},
    {"id":14, "participants": [67], "special":[211]},
    {"id":13, "participants": [8, 11], "special":[23]},
    {"id":15, "participants": [65, 69, 111], "special":[]},
    {"id":21, "participants": [7, 5], "special":[11, 17, 16]},
    {"id":22, "participants": [9, 12], "special":[]},
    {"id":24, "participants": [20, 42], "special":[99]},
    {"id":27, "participants": [8, 54, 75], "special":[78]},
    {"id":33, "participants": [12, 42, 50], "special":[222]},
    {"id":36, "participants": [11, 16, 30, 25, 37], "special":[59]},
]

In [5]:
class CarsDataset(Dataset):
    def __init__(self, root_dir, annotations, transform=None):

        self.transform = transform
        self.frames = []
        self.targets = []
        self.paths = []
        for annot in tqdm(annotations):
            folder = os.path.join(root_dir, str(annot["id"]))
            for file in os.listdir(folder):
                if file == "full.mp4" or not file[-3:]=="mp4":
                    continue
                path = os.path.join(folder, file)
                frames = load_frames(path)
                label = int(file.split(".")[0]) in annot["special"]
                
                if label:
                    targets = [1] * len(frames) 
                    paths = [path] * len(frames) 
                    self.targets.extend(targets)
                    self.frames.extend(frames)
                    self.paths.extend(paths)
                    assert len(frames)  == len(targets) == len(paths)
                    

                else:
                    frames = [f for i, f in enumerate(frames) if i % 25 == 0]
                    targets = [0] * len(frames)
                    paths = [path] * len(frames)

                    assert len(frames) == len(targets) == len(paths)
                    self.targets.extend(targets)
                    self.frames.extend(frames)
                    self.paths.extend(paths)

        
    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx, should_aug=True):
        image = self.frames[idx]
        target = self.targets[idx]
        path = self.paths[idx]
        if self.transform and should_aug:
            image = self.transform(image)
        
        return image, target, path

In [6]:
%env CUDA_VISIBLE_DEVICES=0
!echo $CUDA_VISIBLE_DEVICES



env: CUDA_VISIBLE_DEVICES=0
0


In [7]:
from IPython.display import HTML

import io 
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

show_video('data/det_one_class/4/69.mp4')

In [8]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score

def train_model(model, dataloaders, criterion, optimizer,
                phases, num_epochs=3):
    start_time = time.time()

    acc_history = {k: list() for k in phases}
    loss_history = {k: list() for k in phases}
    best_acc = 0
    attempts = 0
    patience = 5
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            preds = []
            true_labels = []
            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels, paths in tqdm(dataloaders[phase], total=n_batches):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                rocauc_score = 0
                f_score = 0
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    labels = torch.unsqueeze(labels,  0).T.to(torch.float)
                    loss = criterion(outputs, labels)
                    outputs = torch.sigmoid(outputs)
                    labels = labels.T.squeeze().detach().cpu().numpy()
                    outputs = outputs.T.squeeze().detach().cpu().numpy()
                    if len(outputs.shape) == 0:
                        continue
                    preds.extend(outputs)
                    true_labels.extend(labels)
                    
                    

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += np.sum((outputs>0.5) == labels)
            outputs = np.array(preds)
            true_labels = np.array(true_labels)
            r = roc_auc_score(true_labels, outputs, average='weighted')
            fpr, tpr, thresholds = roc_curve(true_labels, outputs)
            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = thresholds[optimal_idx]
            print("Threshold value is:", optimal_threshold)
            print("Roc auc score: ", r)
            print("F-score: ", f1_score(true_labels, outputs > optimal_threshold, average='weighted', zero_division=0))
            
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects
            epoch_acc /= len(dataloaders[phase].dataset)
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,
                                                       epoch_acc))
            if phase == "val" and r > best_acc:
                best_acc = r
                attempts = 0
                torch.save(model.state_dict(), MODEL_WEIGHTS)
            elif phase == "val" and r <= best_acc:
                attempts += 1
                if attempts >= patience:
                    return model, acc_history
        print()

    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                        time_elapsed % 60))

    return model, acc_history

In [9]:
from torchvision import models



In [10]:
from torch import nn, optim
from torch.utils.data import DataLoader



In [11]:
img_size = 224
trans_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
    transforms.RandomRotation(50),
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trans_val = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


ds_train = CarsDataset("data/det_one_class", annot, trans_train)
ds_val = CarsDataset("data/det_one_class", annot, trans_val)

100%|██████████| 17/17 [00:58<00:00,  3.47s/it]


In [12]:
import pandas as pd

data_ds = zip(ds_train.targets, ds_train.paths)

data_ds = pd.DataFrame(data=data_ds, columns=["target", "path"])

print(data_ds.loc[data_ds.target == 1].shape[0], len(data_ds))

5158 12093


In [13]:
valid_set = {"data/det_one_class/36",
            "data/det_one_class/1",
            "data/det_one_class/21",
            "data/det_one_class/22",
            "data/det_one_class/14"}

valid = data_ds[data_ds["path"].apply(lambda x: '/'.join(x.split('/')[:-1]) in valid_set)].index
train = data_ds[data_ds["path"].apply(lambda x: not '/'.join(x.split('/')[:-1]) in valid_set)].index

In [14]:
del data_ds

In [15]:
trainset = torch.utils.data.Subset(ds_train, train)
testset = torch.utils.data.Subset(ds_val, valid)

batch_size = 16
num_workers = 4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)

loaders = {'train': trainloader, 'val': testloader}

In [16]:
class Densenet169(nn.Module):

    def __init__(self):
      super().__init__()
      self.pretrained_model = models.densenet169(pretrained = True)
      
      feature_in = self.pretrained_model.classifier.in_features
      self.pretrained_model.classifier = nn.Linear(feature_in,2)

    def forward(self,x):
      return self.pretrained_model(x)

In [17]:
m = Densenet169()
m.load_state_dict(torch.load("densenet169-2.pt"))

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /home/gorodion/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]

<All keys matched successfully>

In [20]:
m.pretrained_model.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [30]:
import time
MODEL_WEIGHTS = "clf_no_augmentations.pth"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_classes = 1
m = m.to(device)


criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(m.parameters(), lr=1e-4)



train_results = train_model(m, loaders, criterion, optimizer,
                                phases=['train','val'], num_epochs=100)

# This train output is not THE BEST, it is just an experiment

Epoch 0/99
----------


100%|██████████| 569/569 [02:48<00:00,  3.37it/s]


Threshold value is: 0.43039593
Roc auc score:  0.9989004650465954
F-score:  0.9930774808152257
train Loss: 0.0271 Acc: 0.9930


100%|██████████| 188/188 [00:12<00:00, 14.56it/s]


Threshold value is: 0.007930559
Roc auc score:  0.6918138476374631
F-score:  0.8374735884044958
val Loss: 1.2824 Acc: 0.8573

Epoch 1/99
----------


...

Threshold value is: 0.9019433
Roc auc score:  0.6048511717757167
F-score:  0.8112487212819377
val Loss: 2.6021 Acc: 0.8166

Epoch 16/99
----------


100%|██████████| 569/569 [02:41<00:00,  3.52it/s]


Threshold value is: 0.4577767
Roc auc score:  0.9992883542618396
F-score:  0.9953849164694127
train Loss: 0.0201 Acc: 0.9955


100%|██████████| 188/188 [00:13<00:00, 14.15it/s]

Threshold value is: 0.018466717
Roc auc score:  0.6160935229705703
F-score:  0.7926197495445727
val Loss: 2.3548 Acc: 0.8229


In [21]:
m.load_state_dict(torch.load("Densenet169_largeVal.pth"))
m = m.to(device)

In [26]:
true_labels = []
preds = []
data_paths = []
m.eval()
for inputs, labels, paths in tqdm(loaders["val"]):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                rocauc_score = 0
                f_score = 0
                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    outputs = m(inputs)
                    labels = torch.unsqueeze(labels,  0).T.to(torch.float)
                    outputs = torch.sigmoid(outputs)
                    labels = labels.T.squeeze().detach().cpu().numpy()
                    outputs = outputs.T.squeeze().detach().cpu().numpy()
                    if len(outputs.shape) == 0:
                        continue
                    
                    paths = list(paths)
                    preds.extend(outputs)
                    true_labels.extend(labels)
                    data_paths.extend(paths)
                    

100%|██████████| 188/188 [00:11<00:00, 16.89it/s]


In [27]:
import numpy as np
r = roc_auc_score(true_labels, preds, average='weighted')
fpr, tpr, thresholds = roc_curve(true_labels, preds)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Threshold value is:", optimal_threshold)
print("Roc auc score: ", r)
print("F-score: ", f1_score(true_labels, preds > optimal_threshold, average='weighted', zero_division=0))
            

Threshold value is: 0.03819097
Roc auc score:  0.9083611140685091
F-score:  0.824468177763129
